In [ ]:
from sys import path
from pathlib import Path
from sqlalchemy import create_engine

import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [ ]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    # ventas_julio = pd.read_sql_table('ventas_julio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [ ]:
ventas = ventas_enero
vend_id = 227
vend_nom = ventas[ventas['salesperson_id'] == vend_id].iloc[0,15]

print(f'Escogiste a la vendedora "{vend_nom}" para el siguiente estudio de ventas')

Genera una tabla con todas las facturas vendidas en el mes de la vendedora

In [ ]:
# ventas[ventas['salesperson_id'] == vend_id].to_excel(f'Facturas {vend_nom}.xlsx')
ventas[ventas['salesperson_id'] == vend_id].head()

¿Cuál es el total vendido en el mes?

In [ ]:
ventas.loc[ventas['salesperson_id'] == vend_id, ['salesperson_name', 'price_subtotal']].groupby(by='salesperson_name').sum()

¿Cuál es la órden de venta dónde más vendió y cuánto?

In [ ]:
wep = ventas[['invoice_origin','price_subtotal']].groupby('invoice_origin').sum().sort_values(by=['price_subtotal'],ascending=False).iloc[0]

f"{vend_nom} vendió ${wep['price_subtotal']:,.2f} en la orden {wep.name}"

¿Cuáles son los 5 artículos que más vende? Por Cantidad

In [ ]:
wep = ventas[['prod_código','product_name', 'quantity']].groupby('prod_código').sum().sort_values(by=['quantity'],ascending=False).iloc[0:5]
print(f"{vend_nom}")
wep

¿Cuáles son los 5 artículos que más vende? Por precio

In [ ]:
wep = ventas[['prod_código','product_name', 'price_subtotal']].groupby('prod_código').sum().sort_values(by=['price_subtotal'],ascending=False).iloc[0:5]
print(f"{vend_nom}")
wep

¿Cuáles son los mejores 3 clientes de la vendedora? 1ro por cantidad de productos vendidos, 2do por subtotal de venta y el último por cantidad de OV en el mes.

In [ ]:
wep = ventas[['partner_id','quantity','price_subtotal']].groupby('partner_id').agg({'quantity':['sum',],'price_subtotal':['sum',]})

cprod = wep[('quantity', 'sum')].sort_values(ascending=False)
cprod_name = cprod.keys()[0]
cprod_cant = cprod.loc[cprod_name]
csub = wep[('price_subtotal', 'sum')].sort_values(ascending=False)
csub_name = csub.keys()[0]
csub_cant = csub.loc[csub_name]


wep = ventas[['partner_id','invoice_origin']].groupby(['partner_id', 'invoice_origin']).count().reset_index().groupby('partner_id').count()

cov = wep['invoice_origin'].sort_values(ascending=False)
cov_name = cov.keys()[0]
cov_cant = cov.loc[cov_name]

print(f'La vendedora {vend_nom} tiene a sus 3 mejores clientes de la siguiente manera:')
print(f'\tEl cliente {cov_name} es el cliente que más órdenes de venta tiene, siendo un total de {cov_cant:,.2f} órdenes')
print(f'\tEl cliente {cprod_name} es el cliente que más cantidad de producto, siendo un total de {cprod_cant:,.2f} artículos vendidos')
print(f'\tEl cliente {csub_name} es el cliente que más facturó, siendo un total de ${csub_cant:,.2f} pesos en el mes de enero')


Ahora resuelve de nuevo la pregunta anterir, sin tomar al cliente MOSTRADOR

In [ ]:
wep = ventas.loc[ventas['partner_id'] != 16253, ['partner_id','quantity','price_subtotal']].groupby('partner_id').agg({'quantity':['sum',],'price_subtotal':['sum',]})

cprod = wep[('quantity', 'sum')].sort_values(ascending=False)
cprod_name = cprod.keys()[0]
cprod_cant = cprod.loc[cprod_name]
csub = wep[('price_subtotal', 'sum')].sort_values(ascending=False)
csub_name = csub.keys()[0]
csub_cant = csub.loc[csub_name]


wep = ventas.loc[ventas['partner_id'] != 16253, ['partner_id','invoice_origin']].groupby(['partner_id', 'invoice_origin']).count().reset_index().groupby('partner_id').count()

cov = wep['invoice_origin'].sort_values(ascending=False)
cov_name = cov.keys()[0]
cov_cant = cov.loc[cov_name]

print(f'La vendedora {vend_nom} tiene a sus 3 mejores clientes de la siguiente manera:')
print(f'\tEl cliente {cov_name} es el cliente que más órdenes de venta tiene, siendo un total de {cov_cant:,.2f} órdenes')
print(f'\tEl cliente {cprod_name} es el cliente que más cantidad de producto, siendo un total de {cprod_cant:,.2f} artículos vendidos')
print(f'\tEl cliente {csub_name} es el cliente que más facturó, siendo un total de ${csub_cant:,.2f} pesos en el mes de enero')